In [22]:
import cv2
import mediapipe as mp
import time
import pandas as pd
import numpy as np

In [ ]:
MAX_FRAMES_LEN = 300
MAX_FRAMES_NUM = 298

# MediaPipe Pose 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=0,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    smooth_landmarks=True
)
mp_drawing = mp.solutions.drawing_utils

# 중요 관절 인덱스
important_landmarks = [
    0, 11, 12, 13, 14, 15, 16,
    23, 24, 25, 26, 29, 30
]

test_df = pd.DataFrame(columns = ["frame", "landmark_id", "x", "y", "z"])

for frame in np.arange(1, 301):
    for idx in important_landmarks:
        test_df.loc[len(test_df)] = [frame, idx, np.nan, np.nan, np.nan]

# 웹캠에서 영상 받기
cap = cv2.VideoCapture(0)  # 기본 카메라 사용

frame_count = 0
processed_frame = 1
start_time = time.time()
predict_time = start_time

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    current_time = time.time()  
    elapsed = current_time - start_time
    predict_elapsed = current_time - predict_time

    if elapsed >= 0.1:
        start_time = current_time
        # MediaPipe에 넣기 위해 RGB로 변환
        frame = cv2.resize(frame, (640, 480))
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        # 관절 데이터 추출 및 표시
        if results.pose_landmarks:
            for idx in important_landmarks:
                landmark = results.pose_landmarks.landmark[idx]
                x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)

        if results.pose_landmarks:
            for idx in important_landmarks:
                landmark = results.pose_landmarks.landmark[idx]
                test_df[test_df["frame"] == processed_frame] = [processed_frame, idx, landmark.x, landmark.y, landmark.z]

        unique_frames = test_df['frame'].unique()
        
        if len(unique_frames) > MAX_FRAMES_LEN:
            # 가장 오래된 관측 프레임 번호
            oldest_frame = unique_frames[0]
            # 해당 프레임 삭제
            test_df = test_df[test_df['frame'] != oldest_frame].reset_index(drop=True)
            # 전체 포즈를 그려볼 수도 있음
            # mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        if processed_frame < MAX_FRAMES_NUM:
            processed_frame += 3
        
        cv2.imshow('Real-Time Pose Landmarks', frame)

        # Q 누르면 종료
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    elif predict_elapsed >= 5:
        test_df = test_df.groupby("landmark_id").apply(lambda x : x.sort_values(by = ["frame", "landmark_id"]).interpolate().ffill().bfill()).reset_index(drop=True)
        test_df = test_df.sort_values(by = ["frame", "landmark_id"])
        print(test_df + "\n\n")


cap.release()
cv2.destroyAllWindows()


In [55]:
len(test_df["frame"].unique())


300

In [ ]:
test_df.head(30)


,frame,landmark_id,x,y,z
0,1.0,0.0,NaN,NaN,NaN
1,1.0,11.0,NaN,NaN,NaN
2,1.0,12.0,NaN,NaN,NaN
3,1.0,13.0,NaN,NaN,NaN
4,1.0,14.0,NaN,NaN,NaN
...,...,...,...,...,...
3895,300.0,24.0,NaN,NaN,NaN
3896,300.0,25.0,NaN,NaN,NaN
3897,300.0,26.0,NaN,NaN,NaN
3898,300.0,29.0,NaN,NaN,NaN


In [50]:
test_df = pd.DataFrame(columns = ["frame", "landmark_id", "x", "y", "z"])

for frame in np.arange(1, 301):
    for idx in important_landmarks:
        test_df.loc[len(test_df)] = [frame, idx, np.nan, np.nan, np.nan]

test_df

,frame,landmark_id,x,y,z
0,1.0,0.0,NaN,NaN,NaN
1,1.0,11.0,NaN,NaN,NaN
2,1.0,12.0,NaN,NaN,NaN
3,1.0,13.0,NaN,NaN,NaN
4,1.0,14.0,NaN,NaN,NaN
...,...,...,...,...,...
3895,300.0,24.0,NaN,NaN,NaN
3896,300.0,25.0,NaN,NaN,NaN
3897,300.0,26.0,NaN,NaN,NaN
3898,300.0,29.0,NaN,NaN,NaN


In [ ]:
test_df = test_df.groupby("landmark_id").apply(lambda x : x.sort_values(by = ["frame", "landmark_id"]).interpolate().ffill().bfill()).reset_index(drop=True)
test_df = test_df.sort_values(by = ["frame", "landmark_id"])

SyntaxError: invalid syntax (2259856835.py, line 1)